In [16]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

# Load the dataset
df = pd.read_csv("companies.csv")

# Display basic info
print("Dataset shape:", df.shape)
print("\nFirst few rows:")
display(df.head())
print("\nData types and missing values:")
display(df.info())

Dataset shape: (198944, 44)

First few rows:


,id,Unnamed: 0.1,entity_type,entity_id,parent_id,name,normalized_name,permalink,category_code,status,...,first_milestone_at,last_milestone_at,milestones,relationships,created_by,created_at,updated_at,lat,lng,ROI
0,c:1,0,Company,1,NaN,Wetpaint,wetpaint,/company/wetpaint,web,operating,...,2010-09-05,2013-09-18,5.0,17.0,initial-importer,2007-05-25 06:51:27,2013-04-13 03:29:00,47.606209,-122.332071,15.5
1,c:10,1,Company,10,NaN,Flektor,flektor,/company/flektor,games_video,acquired,...,NaN,NaN,NaN,6.0,initial-importer,2007-05-31 21:11:51,2008-05-23 23:23:14,34.021122,-118.396467,NaN
2,c:100,2,Company,100,NaN,There,there,/company/there,games_video,acquired,...,2003-02-01,2011-09-23,4.0,12.0,initial-importer,2007-08-06 23:52:45,2013-11-04 02:09:48,37.562992,-122.325525,NaN
3,c:10000,3,Company,10000,NaN,MYWEBBO,mywebbo,/company/mywebbo,network_hosting,operating,...,NaN,NaN,NaN,NaN,NaN,2008-08-24 16:51:57,2008-09-06 14:19:18,NaN,NaN,NaN
4,c:10001,4,Company,10001,NaN,THE Movie Streamer,the movie streamer,/company/the-movie-streamer,games_video,operating,...,NaN,NaN,NaN,NaN,NaN,2008-08-24 17:10:34,2008-09-06 14:19:18,NaN,NaN,NaN



Data types and missing values:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 198944 entries, 0 to 198943
Data columns (total 44 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   id                   198944 non-null  object 
 1   Unnamed: 0.1         198944 non-null  int64  
 2   entity_type          198944 non-null  object 
 3   entity_id            198944 non-null  int64  
 4   parent_id            0 non-null       float64
 5   name                 198921 non-null  object 
 6   normalized_name      198918 non-null  object 
 7   permalink            198944 non-null  object 
 8   category_code        123891 non-null  object 
 9   status               198944 non-null  object 
 10  founded_at           91769 non-null   object 
 11  closed_at            2632 non-null    object 
 12  domain               127263 non-null  object 
 13  homepage_url         127263 non-null  object 
 14  twitter_username     81042 non-null 

None

In [17]:
# Check for missing values
missing_values = df.isnull().sum()
print("Missing values per column:")
display(missing_values[missing_values > 0])

# Drop columns with too many missing values (more than 50%)
threshold = len(df) * 0.5
df_cleaned = df.dropna(thresh=threshold, axis=1)

# For numerical columns with missing values, fill with median
num_cols = df_cleaned.select_dtypes(include=['int64', 'float64']).columns
for col in num_cols:
    if df_cleaned[col].isnull().sum() > 0:
        df_cleaned[col].fillna(df_cleaned[col].median(), inplace=True)

# For categorical columns with missing values, fill with mode
cat_cols = df_cleaned.select_dtypes(include=['object']).columns
for col in cat_cols:
    if df_cleaned[col].isnull().sum() > 0:
        df_cleaned[col].fillna(df_cleaned[col].mode()[0], inplace=True)

# Verify no missing values remain
print("\nMissing values after cleaning:")
display(df_cleaned.isnull().sum().sum())

Missing values per column:


,0
parent_id,198944
name,23
normalized_name,26
category_code,75053
founded_at,107175
closed_at,196312
domain,71681
homepage_url,71681
twitter_username,117902
logo_url,88209


/tmp/ipython-input-263204603.py:14: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_cleaned[col].fillna(df_cleaned[col].median(), inplace=True)
/tmp/ipython-input-263204603.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned[col].fillna(df_cleaned[col].median(), inplace=True)
/tmp/ipython-input-263204603.py:20: FutureWarning: A va


Missing values after cleaning:


np.int64(0)

In [18]:
# Check for duplicate rows
print("Number of duplicate rows:", df_cleaned.duplicated().sum())

# Remove duplicates
df_cleaned = df_cleaned.drop_duplicates()
print("Shape after removing duplicates:", df_cleaned.shape)

Number of duplicate rows: 2390
Shape after removing duplicates: (196554, 20)


In [19]:
# Convert date columns to datetime
date_cols = ['founded_at', 'closed_at', 'first_investment_at', 'last_investment_at',
             'first_funding_at', 'last_funding_at', 'first_milestone_at',
             'last_milestone_at', 'created_at', 'updated_at']

for col in date_cols:
    if col in df_cleaned.columns:
        df_cleaned[col] = pd.to_datetime(df_cleaned[col], errors='coerce')

# Check numeric columns stored as strings
for col in df_cleaned.columns:
    if df_cleaned[col].dtype == 'object':
        try:
            df_cleaned[col] = pd.to_numeric(df_cleaned[col])
            print(f"Converted {col} to numeric")
        except:
            pass

print("\nData types after conversion:")
display(df_cleaned.dtypes)


Data types after conversion:


,0
id,object
Unnamed: 0.1,int64
entity_type,object
entity_id,int64
name,object
normalized_name,object
permalink,object
category_code,object
status,object
domain,object


In [20]:
# Clean categorical columns by stripping whitespace and standardizing
cat_cols = df_cleaned.select_dtypes(include=['object']).columns
for col in cat_cols:
    df_cleaned[col] = df_cleaned[col].str.strip().str.lower()

# Example: Clean status column
if 'status' in df_cleaned.columns:
    df_cleaned['status'] = df_cleaned['status'].str.strip().str.lower()
    print("Status values after cleaning:", df_cleaned['status'].unique())

Status values after cleaning: ['operating' 'acquired' 'closed' 'ipo']


TASK 2 : Data Manipulation

In [21]:
# Calculate company age
if 'founded_at' in df_cleaned.columns:
    current_year = datetime.now().year
    df_cleaned['founded_year'] = pd.to_datetime(df_cleaned['founded_at']).dt.year
    df_cleaned['company_age'] = current_year - df_cleaned['founded_year']
    # Handle missing/outlier ages
    df_cleaned['company_age'] = df_cleaned['company_age'].apply(lambda x: x if x > 0 and x < 100 else np.nan)
    df_cleaned['company_age'].fillna(df_cleaned['company_age'].median(), inplace=True)

# Extract month and day from dates if needed
for col in ['founded_at', 'closed_at']:
    if col in df_cleaned.columns:
        df_cleaned[f'{col}_month'] = pd.to_datetime(df_cleaned[col]).dt.month
        df_cleaned[f'{col}_day'] = pd.to_datetime(df_cleaned[col]).dt.day

# Create a binary indicator for whether the company has a homepage
if 'homepage_url' in df_cleaned.columns:
    df_cleaned['has_homepage'] = df_cleaned['homepage_url'].notnull().astype(int)
else:
    df_cleaned['has_homepage'] = 0  # Assume no homepage if column doesn't exist

# Create a binary indicator for whether the company has a Twitter account
if 'twitter_username' in df_cleaned.columns:
    df_cleaned['has_twitter'] = df_cleaned['twitter_username'].notnull().astype(int)
else:
    df_cleaned['has_twitter'] = 0

In [22]:
from sklearn.preprocessing import MinMaxScaler

# Select numerical columns to scale
num_cols_to_scale = ['funding_total_usd', 'investment_rounds', 'milestones', 'company_age']
num_cols_to_scale = [col for col in num_cols_to_scale if col in df_cleaned.columns]

if num_cols_to_scale:
    scaler = MinMaxScaler()
    df_cleaned[num_cols_to_scale] = scaler.fit_transform(df_cleaned[num_cols_to_scale])
    print("Scaled numerical columns:", num_cols_to_scale)

In [23]:
from sklearn.preprocessing import LabelEncoder

# Label encode binary categorical variables
binary_cat_cols = ['has_homepage', 'has_twitter']
for col in binary_cat_cols:
    if col in df_cleaned.columns:
        df_cleaned[col] = LabelEncoder().fit_transform(df_cleaned[col])

# One-hot encode other categorical variables
cat_cols_to_encode = ['category_code', 'country_code', 'state_code', 'region']
cat_cols_to_encode = [col for col in cat_cols_to_encode if col in df_cleaned.columns]

if cat_cols_to_encode:
    df_cleaned = pd.get_dummies(df_cleaned, columns=cat_cols_to_encode, drop_first=True)
    print("One-hot encoded columns:", cat_cols_to_encode)

One-hot encoded columns: ['category_code', 'region']


Task 3 :Data Labeling

In [24]:
# Create target variable based on status
if 'status' in df_cleaned.columns:
    # Define acquisition/closed as 0, operating/ipo as 1
    df_cleaned['active_status'] = df_cleaned['status'].apply(
        lambda x: 0 if x in ['acquired', 'closed'] else 1 if x in ['operating', 'ipo'] else np.nan
    )

    # Drop rows where we couldn't determine status
    df_cleaned = df_cleaned.dropna(subset=['active_status'])

    # Convert to integer
    df_cleaned['active_status'] = df_cleaned['active_status'].astype(int)

    print("\nTarget variable distribution:")
    display(df_cleaned['active_status'].value_counts())


Target variable distribution:


,count
active_status,
1,184576
0,11978


Dataset Preparation

In [25]:
# Drop unnecessary columns
cols_to_drop = ['id', 'Unnamed: 0.1', 'entity_type', 'entity_id', 'parent_id',
                'name', 'normalized_name', 'permalink', 'domain', 'homepage_url',
                'twitter_username', 'logo_url', 'short_description', 'description',
                'overview', 'tag_list', 'created_by', 'created_at', 'updated_at']

cols_to_drop = [col for col in cols_to_drop if col in df_cleaned.columns]
df_final = df_cleaned.drop(columns=cols_to_drop)

# Save the cleaned dataset
df_final.to_csv('cleaned_startup_data.csv', index=False)
print("\nFinal dataset shape:", df_final.shape)
display(df_final.head())


Final dataset shape: (196554, 5877)


,status,logo_width,logo_height,relationships,has_homepage,has_twitter,category_code_analytics,category_code_automotive,category_code_biotech,category_code_cleantech,...,region_zurih,region_zutphen,region_zwaag,region_zwijnaarde,region_zwijnaarde / ghent,region_zwijndrecht,region_zwingenberg,region_zwolle,region_zürich-schlieren,active_status
0,operating,401.0,54.0,17.0,0,0,False,False,False,False,...,False,False,False,False,False,False,False,False,False,1
1,acquired,186.0,85.0,6.0,0,0,False,False,False,False,...,False,False,False,False,False,False,False,False,False,0
2,acquired,107.0,34.0,12.0,0,0,False,False,False,False,...,False,False,False,False,False,False,False,False,False,0
3,operating,268.0,105.0,1.0,0,0,False,False,False,False,...,False,False,False,False,False,False,False,False,False,1
4,operating,200.0,74.0,1.0,0,0,False,False,False,False,...,False,False,False,False,False,False,False,False,False,1


Summary Report

In [12]:
# Generate a summary report
report = {
    "original_shape": df.shape,
    "cleaned_shape": df_final.shape,
    "columns_removed": list(set(df.columns) - set(df_final.columns)),
    "missing_values_initial": df.isnull().sum().sum(),
    "missing_values_final": df_final.isnull().sum().sum(),
    "duplicates_removed": df.duplicated().sum(),
    "target_distribution": dict(df_final['active_status'].value_counts()),
    "numerical_features_scaled": num_cols_to_scale,
    "categorical_features_encoded": cat_cols_to_encode
}

print("\nData Preprocessing Summary Report:")
for key, value in report.items():
    print(f"\n{key.replace('_', ' ').title()}:")
    display(value)


Data Preprocessing Summary Report:

Original Shape:


(70897, 44)


Cleaned Shape:


(68507, 3467)


Columns Removed:


...

{1: np.int64(65183), 0: np.int64(3324)}


Numerical Features Scaled:


[]


Categorical Features Encoded:


['category_code', 'region']

In [13]:
# Save the cleaned dataset with active_status column
df_final.to_csv('cleaned_startup_data.csv', index=False)
print("Cleaned dataset saved as 'cleaned_startup_data.csv'")

Cleaned dataset saved as 'cleaned_startup_data.csv'


In [15]:
# Verify the active_status column was added correctly
if 'active_status' in df_final.columns:
    print("\nActive Status Distribution:")
    print(df_final['active_status'].value_counts())

    # Check which columns are available to display
    available_cols = []
    for col in ['name', 'status', 'active_status']:
        if col in df_final.columns:
            available_cols.append(col)

    if len(available_cols) > 1:  # Need at least status and active_status
        print("\nSample of data with active_status:")
        display(df_final[available_cols].head())
    else:
        print("\nCould not display sample - missing required columns")
else:
    print("Error: active_status column was not created successfully")


Active Status Distribution:
active_status
1    65183
0     3324
Name: count, dtype: int64

Sample of data with active_status:


,status,active_status
0,operating,1
1,acquired,0
2,acquired,0
3,operating,1
4,operating,1


from matplotlib import pyplot as plt
_df_0['active_status'].plot(kind='hist', bins=20, title='active_status')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_1.groupby('status').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_2['active_status'].plot(kind='line', figsize=(8, 4), title='active_status')
plt.gca().spines[['top', 'right']].set_visible(False)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_3['status'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_3, x='active_status', y='status', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)